# Simplification pipeline

This is an abstracted simplification pipeline that can be used to resolve all artifacts.

To deal with some minor consequences of simplification, we will need to run this twice, in some way.

Also, it should be combined togehter to a single function that does not call `simplify_singletons`, `simplify_pairs`, and `simplify_clusters` in a sequence like this as there's some duplicated computation hapenning. But it works. Mostly.

In [1]:
import logging
import warnings

import folium
from libpysal import graph

from core import algorithms, utils


Filter out the RuntimeWarning showing on Apple Silicon.

In [2]:
warnings.filterwarnings(
    "ignore",
    category=RuntimeWarning,
    message="invalid value encountered in intersection",
)

Set logging level to debug to see the debugging messages.

In [3]:
# # Get the logger for core.algorithms.simplify
# logger = logging.getLogger('core.algorithms.simplify')
# logger.setLevel(logging.DEBUG)

# # Set the logging format
# formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# # Create a handler for the logger
# handler = logging.StreamHandler()
# handler.setLevel(logging.DEBUG)
# handler.setFormatter(formatter)

# # Add the handler to the logger
# logger.addHandler(handler)

Specify case metadata

In [4]:
case = "Liège"

Read road data

In [5]:
roads = utils.read_no_degree_2(case)

Merge nearby nodes (up to double of distance used in skeleton).

In [6]:
roads = algorithms.simplify.consolidate_nodes(roads, tolerance=2.1)

Identify artifacts

In [7]:
artifacts, _ = algorithms.simplify.get_artifacts(roads)

Remove edges fully within the artifact (dangles).

In [8]:
_, r_idx = roads.sindex.query(artifacts.geometry, predicate="contains")
roads = roads.drop(roads.index[r_idx])
roads = algorithms.simplify.remove_false_nodes(roads)

Filter singleton artifacts

In [9]:
rook = graph.Graph.build_contiguity(artifacts, rook=True)

# keep only those artifacts which occur as isolates, i.e. are not part of a larger intersection
singles = artifacts.loc[artifacts.index.intersection(rook.isolates)]

Filter doubles

In [10]:
artifacts["comp"] = rook.component_labels
counts = artifacts["comp"].value_counts()
doubles = artifacts.loc[artifacts["comp"].isin(counts[counts == 2].index)]

Filter clusters

In [11]:
clusters = artifacts.loc[artifacts["comp"].isin(counts[counts > 2].index)].copy()

Simplify artifacts.

In [12]:
new_roads = algorithms.simplify.simplify_singletons(singles, roads)
new_roads = algorithms.simplify.simplify_pairs(doubles, new_roads)
new_roads = algorithms.simplify.simplify_clusters(clusters, new_roads)

/Users/martin/Git/simplification/core/geometry.py:189: UserWarning: Could not create a connection as it would lead outside of the artifact.
  additions, splits = snap_to_targets(
/Users/martin/miniforge3/envs/simplification/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)
/Users/martin/miniforge3/envs/simplification/lib/python3.11/site-packages/geopandas/geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

In [ ]:
new_roads.explore()